# Athena setup

This code is only supossed to be ran once, after that the tables and views created are upadted automatically 

In [ ]:
import boto3
import time

DATABASE_NAME = 'ambrus'
S3_OUTPUT = 's3://business-news-sentiments/athena-temp-results/'

In [ ]:
def create_athena_view(view_name, database, sql_query, output_location):
    """
    Creates an Athena view using Boto3.
    """
    client = boto3.client('athena')

    # 1. Construct the Query
    # It is best practice to use "CREATE OR REPLACE VIEW" to handle updates easily
    create_view_sql = f"""
    CREATE OR REPLACE VIEW "{database}"."{view_name}" AS 
    {sql_query}
    """

    print(f"Creating view '{view_name}' in database '{database}'...")

    # 2. Execute the Query
    try:
        response = client.start_query_execution(
            QueryString=create_view_sql,
            QueryExecutionContext={
                'Database': database
            },
            ResultConfiguration={
                'OutputLocation': output_location
            }
        )
        
        query_execution_id = response['QueryExecutionId']
        print(f"Execution ID: {query_execution_id}")

    except Exception as e:
        print(f"Error starting query execution: {e}")
        return

    # 3. Wait for the query to complete (Optional but recommended)
    state = 'RUNNING'
    while state in ['RUNNING', 'QUEUED']:
        response = client.get_query_execution(QueryExecutionId=query_execution_id)
        state = response['QueryExecution']['Status']['State']
        
        if state == 'FAILED':
            reason = response['QueryExecution']['Status']['StateChangeReason']
            print(f"View creation FAILED: {reason}")
            return
        elif state == 'CANCELLED':
            print("View creation CANCELLED.")
            return
        
        time.sleep(1) # Wait a second before checking again

    print(f"View '{view_name}' created successfully!")

# The SELECT statement that defines your view logic
# NOTE: Do not include the 'CREATE VIEW' part here, just the SELECT logic
cnn_sentiment_clean = """
SELECT
  title
, TRY_CAST(date AS DATE) clean_date
, sentiment
, topic_inflation
, topic_taxes
, topic_stocks
, topic_jobs
, topic_housing
, topic_energy
, topic_crypto
FROM
  ambrus.cnn_sentiment
WHERE ((NOT (link LIKE '%cnn-underscored%')) AND (link LIKE '%/20__/%') AND (sentiment IS NOT NULL))
"""
fox_sentiment_clean = """
SELECT
  title
, TRY_CAST(date AS DATE) clean_date
, sentiment
, topic_inflation
, topic_taxes
, topic_stocks
, topic_jobs
, topic_housing
, topic_energy
, topic_crypto
FROM
  ambrus.fox_sentiment
WHERE (sentiment IS NOT NULL)
"""


monthly_combined_sentiments = """
WITH
  cnn_monthly AS (
   SELECT
     date_trunc('month', "clean_date") month_start
   , (((COUNT(*) - SUM("sentiment")) * 100) / COUNT(*)) cnn_score
   FROM
     "ambrus"."cnn_sentiment_clean"
   GROUP BY 1
) 
, fox_monthly AS (
   SELECT
     date_trunc('month', "clean_date") month_start
   , (((COUNT(*) - SUM("sentiment")) * 100) / COUNT(*)) fox_score
   FROM
     "ambrus"."fox_sentiment_clean"
   GROUP BY 1
) 
SELECT
  date_format(COALESCE(c.month_start, f.month_start), '%Y-%m') month_year
, COALESCE(c.cnn_score, 0) cnn_sentiment
, COALESCE(f.fox_score, 0) fox_sentiment
FROM
  (cnn_monthly c
FULL JOIN fox_monthly f ON (c.month_start = f.month_start))
ORDER BY 1 DESC
"""

monthly_topic_breakdown = """
WITH
  combined_data AS (
   SELECT
     *
   , 'CNN' news_site
   FROM
     "ambrus"."cnn_sentiment_clean"
UNION ALL    SELECT
     *
   , 'FOX' news_site
   FROM
     "ambrus"."fox_sentiment_clean"
) 
SELECT
  date_format("clean_date", '%Y-%m') month_year
, news_site
, ((SUM(topic_inflation) * 100) / COUNT(*)) topic_inflation_sum
, ((SUM(topic_taxes) * 100) / COUNT(*)) topic_taxes_sum
, ((SUM(topic_stocks) * 100) / COUNT(*)) topic_stocks_sum
, ((SUM(topic_jobs) * 100) / COUNT(*)) topic_jobs_sum
, ((SUM(topic_housing) * 100) / COUNT(*)) topic_housing_sum
, ((SUM(topic_energy) * 100) / COUNT(*)) topic_energy_sum
, ((SUM(topic_crypto) * 100) / COUNT(*)) topic_crypto_sum
FROM
  combined_data
GROUP BY 1, 2
"""

In [ ]:
def create_athena_table(database, create_table_sql, output_location):
    client = boto3.client('athena')

    print(f"Creating table in database '{database}'...")

    try:
        # Execute the DDL
        response = client.start_query_execution(
            QueryString=create_table_sql,
            QueryExecutionContext={'Database': database},
            ResultConfiguration={'OutputLocation': output_location}
        )
        query_id = response['QueryExecutionId']
        print(f"Execution ID: {query_id}")

        # Wait for completion
        while True:
            stats = client.get_query_execution(QueryExecutionId=query_id)
            status = stats['QueryExecution']['Status']['State']
            
            if status in ['SUCCEEDED']:
                print("Table created successfully.")
                break
            elif status in ['FAILED', 'CANCELLED']:
                reason = stats['QueryExecution']['Status']['StateChangeReason']
                raise Exception(f"Table creation failed: {reason}")
            
            time.sleep(1)

    except Exception as e:
        print(e)

# Define your table SQL here (See examples below)
# Note: Ensure your S3 LOCATION ends with a forward slash '/'
fox_sentiment = """
CREATE EXTERNAL TABLE
ambrus.fox_sentiment (
    title STRING,
    link STRING,          
    date DATE,
    is_economy BOOLEAN,
    sentiment INT,
    topic_inflation INT,
    topic_taxes INT,
    topic_stocks INT,
    topic_jobs INT,
    topic_housing INT,
    topic_energy INT,
    topic_crypto INT)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
LOCATION 's3://business-news-sentiments/news_sentiments/fox';
"""

cnn_sentiment = """
CREATE EXTERNAL TABLE ambrus.cnn_sentiment (
    title STRING,
    link STRING,          
    date STRING,
    is_economy BOOLEAN,         
    sentiment INT,
    topic_inflation INT,
    topic_taxes INT,
    topic_stocks INT,
    topic_jobs INT,
    topic_housing INT,
    topic_energy INT,
    topic_crypto INT
)
ROW FORMAT SERDE 'org.openx.data.jsonserde.JsonSerDe'
LOCATION 's3://business-news-sentiments/news_sentiments/cnn';
"""

In [ ]:
create_athena_table(DATABASE_NAME, fox_sentiment, S3_OUTPUT)
create_athena_table(DATABASE_NAME, cnn_sentiment, S3_OUTPUT)

In [ ]:
create_athena_view("cnn_sentiment_clean_test", DATABASE_NAME, cnn_sentiment_clean, S3_OUTPUT)
create_athena_view("fox_sentiment_clean_test", DATABASE_NAME, fox_sentiment_clean, S3_OUTPUT)
create_athena_view("monthly_combined_sentiments_test", DATABASE_NAME, monthly_combined_sentiments, S3_OUTPUT)
create_athena_view("monthly_topic_breakdown_test", DATABASE_NAME, monthly_topic_breakdown, S3_OUTPUT)